# TAREA FINAL: Datos de vivienda

<p> El negocio inmobiliario es uno de los motores de la economía de España, el carácter turístico de nuesta península hace que sean muchos los inversores extranjeros que se decidan por buscar una inversión de bajo riesgos en nuestras ciudades costeras o grandes urbes. 
Muchas de estas inversiones se hacen sobre grandes bolsas de inmuebles que deben ser analizados previamente para comprobar la rentabilidad del porfolio </p>

<!-- <p> En este caso vamos a trabajar con una tabla que contienen información de distintos inmuebles repartidos por una zona específica, sus carácterísticas y su precio </p>  -->

Todas las cuestiones se deben realizar sobre el conjunto de casos que representen viviendas ubicadas en zonas residenciales **(alta, media y baja densidad)**

**MUY IMPORTANTE:** En las otras prácticas he detectado colaboraciones involucrando varias personas y he sido flexible aunque a algunos os lo he mencionado en las correcciones, porque al final el trabajo de analista de datos es un trabajo colaborativo. Sin embargo, este trabajo es individual, así que cuidado con las colaboraciones.

Las variables de las que se compone el dataset son:

|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje|||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

Recomiendo al leer los datos, eliminar espacios de los nombres de las columnas, realiza un pequeño análisis inicial de los mismos. No olvides fijarte en los tipos de variables, que variables pueden tener tipos confundidos y corrige los. Sobre todo, trabaja con las fechas.

**NOTA:** Las tareas complementarias sirven para subir nota. El resto de preguntas valen igual y suman 10 puntos.

## Inicializar y cargar el contexto spark

In [389]:
     # Initialize the spark context
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

    # Due to we are going to work with sparkSQL we will introduce the sparksql context
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [390]:
raw_data = spark.read.format("csv") \
                       .option("sep", ";") \
                       .option("inferSchema", "true") \
                       .option("header", "true") \
                       .load('BDpracticafinalCSV.csv') 
print(type(raw_data))

<class 'pyspark.sql.dataframe.DataFrame'>


In [391]:
     # The first 5 observations with take function
raw_data.take(1)

[Row(Order=1, MS Zoning='RL', Lot Frontage=141, Lot Area=31770, Land Contour='Lvl', Land Slope='Gtl', Overall Qual=6, Year Built=1960, Year Remod/Add=1960, Mas Vnr Type='Stone', Exter Qual='TA', Bsmt Cond='Gd', Total Bsmt SF=1080, Heating QC='Fa', Central Air='Y', Full Bath=1, Half Bath=0, Bedroom AbvGr=3, Kitchen AbvGr=1, Kitchen Qual='TA', TotRms AbvGrd=7, Garage Cars=2, Garage Area=528, Garage Cond='TA', Pool Area=0, Pool QC='Sp', Mo Sold=5, Yr Sold=2010, SalePrice=215000)]

In [392]:
    # count the observations
raw_data.count()

2936

In [393]:
    # Le pedimos la primera fila de observaciones con show(1)
raw_data.show(1)

+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|Order|MS Zoning|Lot Frontage|Lot Area|Land Contour|Land Slope|Overall Qual|Year Built|Year Remod/Add|Mas Vnr Type|Exter Qual|Bsmt Cond|Total Bsmt SF|Heating QC|Central Air|Full Bath|Half Bath|Bedroom AbvGr|Kitchen AbvGr|Kitchen Qual|TotRms AbvGrd|Garage Cars|Garage Area|Garage Cond|Pool Area|Pool QC|Mo Sold|Yr Sold|SalePrice|
+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|    1|      

In [394]:
  # Show the schema 
raw_data.printSchema()

root
 |-- Order: integer (nullable = true)
 |-- MS Zoning: string (nullable = true)
 |-- Lot Frontage: integer (nullable = true)
 |-- Lot Area: integer (nullable = true)
 |-- Land Contour: string (nullable = true)
 |-- Land Slope: string (nullable = true)
 |-- Overall Qual: integer (nullable = true)
 |-- Year Built: integer (nullable = true)
 |-- Year Remod/Add: integer (nullable = true)
 |-- Mas Vnr Type: string (nullable = true)
 |-- Exter Qual: string (nullable = true)
 |-- Bsmt Cond: string (nullable = true)
 |-- Total Bsmt SF: integer (nullable = true)
 |-- Heating QC: string (nullable = true)
 |-- Central Air: string (nullable = true)
 |-- Full Bath: integer (nullable = true)
 |-- Half Bath: integer (nullable = true)
 |-- Bedroom AbvGr: integer (nullable = true)
 |-- Kitchen AbvGr: integer (nullable = true)
 |-- Kitchen Qual: string (nullable = true)
 |-- TotRms AbvGrd: integer (nullable = true)
 |-- Garage Cars: integer (nullable = true)
 |-- Garage Area: integer (nullable = tru

Una vez que revisamos que el esquema del fichero sea el correcto para cada una de las variables pasamos a chequear los tipos de residencias que hay para luego filtrar aquellas que nos interesan

In [395]:
raw_data.select("MS Zoning").distinct().show()

+---------+
|MS Zoning|
+---------+
|       RH|
|       FV|
|        C|
|        A|
|       Rl|
|       RL|
|       rL|
|        I|
|       RM|
+---------+



Aqui podemos observar que hay diferentes tipos RL donde algunas letras estan en minusculas y otras en mayusculas. Por lo tanto, corregiremos esos nombres y luego pasaremos a filtrar por los tipos de zonas que nos interesen

In [396]:
    # Utilizamos la funcion withColumn para cambiar las filas mal nombradas y luego con regexp_replace buscamos el nombre que queremos cambiar
raw_data = raw_data.withColumn('MS Zoning', regexp_replace('MS Zoning', 'Rl', 'RL'))
raw_data = raw_data.withColumn('MS Zoning', regexp_replace('MS Zoning', 'rL', 'RL'))

In [397]:
    # Chequeamos los resultados
raw_data.select("MS Zoning").distinct().show()

+---------+
|MS Zoning|
+---------+
|       RH|
|       FV|
|        C|
|        A|
|       RL|
|        I|
|       RM|
+---------+



In [398]:
    # Filtramos por viviendas ubicadas en zonas residenciales (alta, media y baja densidad)
data_filter = raw_data.filter((raw_data["MS Zoning"] == "RM") | (raw_data["MS Zoning"] == "RH") | (raw_data["MS Zoning"] == "RL"))
data_filter.show(1)

+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|Order|MS Zoning|Lot Frontage|Lot Area|Land Contour|Land Slope|Overall Qual|Year Built|Year Remod/Add|Mas Vnr Type|Exter Qual|Bsmt Cond|Total Bsmt SF|Heating QC|Central Air|Full Bath|Half Bath|Bedroom AbvGr|Kitchen AbvGr|Kitchen Qual|TotRms AbvGrd|Garage Cars|Garage Area|Garage Cond|Pool Area|Pool QC|Mo Sold|Yr Sold|SalePrice|
+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|    1|      

In [399]:
data_filter.count()

2768

Revisamos ahora los valores nulos

In [400]:
data_filter.select([count(when(col(c).isNull(), c)).alias(c) for c in 
           data_filter.columns]).show()

+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|Order|MS Zoning|Lot Frontage|Lot Area|Land Contour|Land Slope|Overall Qual|Year Built|Year Remod/Add|Mas Vnr Type|Exter Qual|Bsmt Cond|Total Bsmt SF|Heating QC|Central Air|Full Bath|Half Bath|Bedroom AbvGr|Kitchen AbvGr|Kitchen Qual|TotRms AbvGrd|Garage Cars|Garage Area|Garage Cond|Pool Area|Pool QC|Mo Sold|Yr Sold|SalePrice|
+-----+---------+------------+--------+------------+----------+------------+----------+--------------+------------+----------+---------+-------------+----------+-----------+---------+---------+-------------+-------------+------------+-------------+-----------+-----------+-----------+---------+-------+-------+-------+---------+
|    0|      

Luego de revisar los valores nulos podemos observar que la unica variable con un numero significativo de nulos es Lot Frontage pero el resto de variable tiene un numero practicamente insignificante de valores NaN que no nos afectaran en nuestro análisis.

## Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?

Observamos los valores distintos

In [401]:
data_filter.distinct().count()

2762

In [402]:
data_filter.count()

2768

El dataframe tiene 6 valores duplicados. Desde mi punto de vista, los duplicados se refieren a personas que compraron dos o mas apartamentos de la misma edificacion o en otra edificacion con condiciones similares. Estos duplicados podrian ser eliminados del analisis.

In [403]:
data_clean = data_filter.dropDuplicates()
data_clean.count()

2762

## ¿Podrías decirme el total de inmuebles y el precio medio (Sale Price) de cada zona (MS Zoning)?

Para responder a esta pregunta vamos a seleccionar las tres zonas residenciales de alta, media y baja densidad que estamos analizando junto con el precio de venta. Luego, agruparemos por las zonas, haremos el conteo de los inmuebles en cada una de estas y calcularemos la media por zona con la funcion `agg` para calcular funciones de agrugados. Finalmente con withColumnRenamed vamos a cambiar la columna del conteo por un nombre mas representativo.

In [404]:
data_inmuebles = data_clean.select("MS Zoning", "SalePrice") \
    .groupBy("MS Zoning") \
    .agg(count("MS Zoning"), avg("SalePrice"))\
    .withColumnRenamed("count(MS Zoning)", "num_inmuebles")

data_inmuebles.show()

+---------+-------------+------------------+
|MS Zoning|num_inmuebles|    avg(SalePrice)|
+---------+-------------+------------------+
|       RH|           27|136419.77777777778|
|       RL|         2273|191283.25164980203|
|       RM|          462|126781.39393939394|
+---------+-------------+------------------+



Aqui podemos observar que las zonas de baja densidad poblacional `RL` tienen el mayor numero de inmuebles y el precio de venta promedio mas alto comparado con las demas zonas residenciales.

## Media de Total Bsmt SF por cada década de construcción calculada a partir de Year Built. 

Para responder a esta pregunta voy a realizar primero un chequeo de los primeros y los ultimos años en los que se contruyo para poder agrupar con la clausula case when. Realizado este proceso, voy a seleccionar la agrupacion realizada en el `case when` junto con la superficie del sótano en pies cuadrados `Total Bsmt SF` y calculare el promedio de Total Bsmt agrupando y ordenando por las decadas que he creado.

In [405]:
    # chequeo minimo y maximo año de construccion
data_clean.select(min("Year Built"), max("Year Built")).show()

+---------------+---------------+
|min(Year Built)|max(Year Built)|
+---------------+---------------+
|           1872|           2010|
+---------------+---------------+



In [422]:
    # Con withColumn y expr hago uso de case when para realizar una nueva columna de agrupamiento
df1 = data_clean.withColumn("decadas", 
      expr("case when `Year Built` between 1870 and 1879 then 'Años 1870' " +
                       "when `Year Built` between 1880 and 1889 then 'Años 1880' " +
                       "when `Year Built` between 1890 and 1899 then 'Años 1890' " +
                       "when `Year Built` between 1900 and 1909 then 'Años 1900' " + 
                       "when `Year Built` between 1910 and 1919 then 'Años 1910' " +
                       "when `Year Built` between 1920 and 1929 then 'Años 1920' " +
                       "when `Year Built` between 1930 and 1939 then 'Años 1930' " +
                       "when `Year Built` between 1940 and 1949 then 'Años 1940' " +
                       "when `Year Built` between 1950 and 1959 then 'Años 1950' " +
                       "when `Year Built` between 1960 and 1969 then 'Años 1960' " + 
                       "when `Year Built` between 1970 and 1979 then 'Años 1970' " +
                       "when `Year Built` between 1980 and 1989 then 'Años 1980' " +
                       "when `Year Built` between 1990 and 1999 then 'Años 1990' " +
                       "when `Year Built` between 2000 and 2009 then 'Años 2000' " +
                       "when `Year Built` between 2010 and 2019 then 'Años 2010' " +
                       "end"))

In [423]:
    # Finalmente realizamos la query que nos interesa a los fines del enunciado
data_superficie = df1.select("Total Bsmt SF", "decadas") \
    .groupBy("decadas") \
    .agg(avg("Total Bsmt SF")) \
    .orderBy("decadas")

data_superficie.show()

+---------+------------------+
|  decadas|avg(Total Bsmt SF)|
+---------+------------------+
|Años 1870|             761.0|
|Años 1880|             836.0|
|Años 1890| 885.5833333333334|
|Años 1900| 720.1388888888889|
|Años 1910| 788.3009708737864|
|Años 1920| 833.9368421052632|
|Años 1930| 769.9158878504672|
|Años 1940| 706.9261744966443|
|Años 1950| 968.2955223880597|
|Años 1960| 1082.672268907563|
|Años 1970| 952.7225274725274|
|Años 1980|            1086.0|
|Años 1990|1138.8083067092653|
|Años 2000| 1348.469079939668|
|Años 2010|            1539.0|
+---------+------------------+



## ¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

Para responder a esta pregunta voy a seleccionar las decadas creadas anteriormente junto con Heating QC. Agruparemos por estas dos variables categoricas y luego, con una funcion de agregado como el conteo voy a contar la califaccion de las construcciones para cada decada. Al final, ordenamos por decada y por conteo para que queden bien presentados los resultados.

In [424]:
data_calefaccion = df1.select("Heating QC", "decadas") \
    .groupBy("decadas", "Heating QC") \
    .agg(count("Heating QC")) \
    .orderBy("decadas", "count(Heating QC)")

data_calefaccion.show(100)

+---------+----------+-----------------+
|  decadas|Heating QC|count(Heating QC)|
+---------+----------+-----------------+
|Años 1870|        Fa|                1|
|Años 1870|        TA|                1|
|Años 1870|        Gd|                1|
|Años 1880|        Ex|                2|
|Años 1880|        TA|                3|
|Años 1880|        Gd|                3|
|Años 1890|        Gd|                4|
|Años 1890|        TA|                4|
|Años 1890|        Ex|                4|
|Años 1900|        Fa|                1|
|Años 1900|        Gd|               11|
|Años 1900|        TA|               12|
|Años 1900|        Ex|               12|
|Años 1910|        Fa|                9|
|Años 1910|        Gd|               28|
|Años 1910|        TA|               28|
|Años 1910|        Ex|               38|
|Años 1920|        Po|                1|
|Años 1920|        Fa|               20|
|Años 1920|        Gd|               43|
|Años 1920|        Ex|               51|
|Años 1920|     

Del dataframe presentado arriba podemos observar que las viviendas mejor acondicionadas para el frio son las construidas en los años 2000 porque tienen el mayor numero de Ex que significa Excelente

## ¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?

Aqui voy a seleccionar el precio de venta, el año de la venta, el Lot Area (medido en pies cuadrados) y luego haciendo un cociente entre el precio de venta y el Lot Area por el numero 10.76 para convertirlo a metros cuadrados podemos conseguir visualizar el precio por metro cuadrado. A continuacion, filtramos solo los diez inmuebles con precio por metro cuadrado mas elevado del año 2009

In [409]:
    # Creo una tabla temporal para hacer la query SQL
data_clean.createOrReplaceTempView("table1")
    
    # Realizamos la query
df1 = spark.sql("""
                SELECT `SalePrice`, `Yr Sold`, `Lot Area`, (`SalePrice` / (`Lot Area` / 10.7642626)) AS Precio_por_metrocuadrado
                FROM table1 
                WHERE `Yr Sold` = 2009 
                ORDER BY Precio_por_metrocuadrado DESC
                """)
    # Presento los resultados
df1.show(10)

+---------+-------+--------+------------------------+
|SalePrice|Yr Sold|Lot Area|Precio_por_metrocuadrado|
+---------+-------+--------+------------------------+
|   124000|   2009|    1300|    1026.745048000290...|
|   137000|   2009|    1488|    991.0645001346164...|
|   173000|   2009|    2268|    821.0835228395432...|
|   121000|   2009|    1612|    807.9874532258410...|
|   119500|   2009|    1680|    765.6722504165740...|
|   118000|   2009|    1680|    756.0613016665752...|
|   112000|   2009|    1680|    717.6175066665798...|
|   115000|   2009|    1733|    714.3047888057206...|
|   111000|   2009|    1680|    711.2102074999139...|
|   148500|   2009|    2280|    701.0934193420441...|
+---------+-------+--------+------------------------+
only showing top 10 rows



## Media anual por zonas del precio de venta y metros cuadrados.

Para hacer frente a esta pregunta lo que voy a realizar es una seleccion de la variable correspondiente a la zona, al año de venta, el promedio del precio de venta y el promedio de los metros cuadrados. De nuevo, para conseguir los metros cuadrados voy a dividir por 10.76 para obtener la equivalencia. Despues agrupo y ordeno por zona y por año de venta y presentare los resultados.

In [410]:
    # creamos una tabla temporal
data_clean.createOrReplaceTempView("table2")
    
    # realizo la query SQL
df2 = spark.sql("""
                SELECT `MS Zoning`, `Yr Sold`, AVG(SalePrice), AVG(`Lot Area` / 10.7642626) AS Metros_cuadrados
                FROM table2 
                GROUP BY `MS Zoning`, `Yr Sold` 
                ORDER BY `MS Zoning`, `Yr Sold`  ASC
                """)
df2.show()

+---------+-------+------------------+-------------------+
|MS Zoning|Yr Sold|    avg(SalePrice)|   Metros_cuadrados|
+---------+-------+------------------+-------------------+
|       RH|   2006|140316.66666666666| 659.54355294158000|
|       RH|   2007|          147250.0| 651.29867790481000|
|       RH|   2008|150333.33333333334| 766.11533675020000|
|       RH|   2009|          116206.8| 647.12282288616000|
|       RH|   2010|          126280.0| 753.01024335844000|
|       RL|   2006|194350.64065708418|1043.75992785845000|
|       RL|   2007|193399.98014440434|1059.64373198233000|
|       RL|   2008|189885.58515283844|1047.61951165928000|
|       RL|   2009|190455.55086372362|1002.15171118745000|
|       RL|   2010|  184978.418972332|1011.32666261328000|
|       RM|   2006|127502.63725490196| 574.99544472135000|
|       RM|   2007|128947.62105263158| 625.42822805256000|
|       RM|   2008| 133336.1219512195| 623.64903204976000|
|       RM|   2009|118759.86904761905| 552.5382357976500

## ¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?

En este apartado voy a filtrar por las casas de revestimiento de piedra y las de ladrillo que hacen referencia a los tipos `BrkCmn`, `BrkFace` y `Stone`. Luego de haber filtrado voy a crear una query que agrupe por estos tipos de revestimiento y calcule el promedio recaudado para poder compararlos.

In [411]:
    # primero observamos los valores distintos
data_clean.select("Mas Vnr Type").distinct().show()

+------------+
|Mas Vnr Type|
+------------+
|        None|
|      BrkCmn|
|        null|
|       Stone|
|     BrkFace|
|      CBlock|
+------------+



In [412]:
data_revestimiento = data_clean.filter((data_clean["Mas Vnr Type"] == "BrkCmn") | (data_clean["Mas Vnr Type"] == "BrkFace") | (data_clean["Mas Vnr Type"] == "Stone"))
data_revestimiento.count()

1102

In [413]:
    # creamos una tabla temporal en base al dataset con los revestimientos que nos interesan
data_revestimiento.createOrReplaceTempView("table3")
    
    # hacemos la query SQL
df3 = spark.sql("""
                SELECT `Mas Vnr Type`, AVG(SalePrice)
                FROM table3 
                GROUP BY `Mas Vnr Type`
                ORDER BY `Mas Vnr Type` ASC
                """)
df3.show()

+------------+------------------+
|Mas Vnr Type|    avg(SalePrice)|
+------------+------------------+
|      BrkCmn|          140199.0|
|     BrkFace|210981.79363207548|
|       Stone| 262775.0087336245|
+------------+------------------+



In [428]:
# creamos una tabla temporal en base al dataset con los revestimientos que nos interesan
data_revestimiento.createOrReplaceTempView("table4")
    
    # hacemos la query SQL
df4 = spark.sql("""
                SELECT `Mas Vnr Type`, SUM(SalePrice)
                FROM table4 
                GROUP BY `Mas Vnr Type`
                ORDER BY `Mas Vnr Type` ASC
                """)
df4.show()

+------------+--------------+
|Mas Vnr Type|sum(SalePrice)|
+------------+--------------+
|      BrkCmn|       3504975|
|     BrkFace|     178912561|
|       Stone|      60175477|
+------------+--------------+



Podemos observar como las casas con revestimiento de ladrillo tienen una recaudacion significativa respecto de las viviendas de piedra. No obstante, las viviendas de revestimiento de piedra son mas caras en promedio respecto de las casas con revestimiento de ladrillo. Las de piedra tienen precios en un 87%  superiores respecto de las de ladrillo normal y un 25% superior de las de ladrillo visto pero las de Ladrillo recaudan mas porque evidentemente son mayor cantidad que las de piedra.

## ¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos

Para responder a esta ultima pregunta obligatoria voy a hacer uso de la clausula case when utilizada comunmente en SQL para poder responder. Aqui creare una nueva columna que contenga en el grupo 1 a las casas con dos cocinas y mas de dos plazas de garage y a un grupo dos compuesto por las viviendas con una cocina y una plaza de garage. Luego de hacer esto, voy a seleccionar el precio de venta promedio junto con el grupo para poder compararlos y poder verificar cuanto son mas caras unas viviendas de otras.

In [414]:
# primero observamos los valores distintos
data_clean.select("Kitchen AbvGr").distinct().show()

+-------------+
|Kitchen AbvGr|
+-------------+
|            1|
|            3|
|            2|
+-------------+



In [415]:
data_clean.select("Garage Cars").distinct().show()

+-----------+
|Garage Cars|
+-----------+
|       null|
|          1|
|          3|
|          5|
|          4|
|          2|
|          0|
+-----------+



In [426]:
df4 = data_clean.withColumn("Grupo", 
      expr("case when `Kitchen AbvGr` = 2 and `Garage Cars` >= 2 then 'Grupo 1' " +
                       "when `Kitchen AbvGr` = 1 and `Garage Cars` = 1 then 'Grupo 2' " +
                       "end"))

In [427]:
data_equipamiento = df4.select("SalePrice", "Grupo") \
    .filter(df4.Grupo.isNotNull()) \
    .groupBy("Grupo") \
    .agg(avg("SalePrice")) \
    .orderBy("Grupo")

data_equipamiento.show()

+-------+------------------+
|  Grupo|    avg(SalePrice)|
+-------+------------------+
|Grupo 1|145124.27906976745|
|Grupo 2|128121.99337748345|
+-------+------------------+



Se puede observar que las viviendas del primer grupo son superiores en un 13% en precio promedio que las del grupo 2, por lo tanto, concluimos que tiene significancia el hecho de tener mas de una cocina o de una plaza de garage

## (COMPLEMENTARIA) Estudiar la relación entre el precio y el número de Garajes. (Recomiendo segmentar precio por cuartiles y estudiar el número de casos coincidentes)

Para responder voy a investigar los minimos, maximos y los cuartiles del precio de venta. Una vez que tenemos estos valores identificados, voy a averiguar en que parte de la distribucion del precio cae el numero de garages. Es decir, averiguaremos si las viviendas de un garage caen en el primer cuartil o en el segundo, etc.

In [418]:
data_clean.describe("SalePrice").show()

+-------+------------------+
|summary|         SalePrice|
+-------+------------------+
|  count|              2762|
|   mean|179957.70057929037|
| stddev|  80219.2667393635|
|    min|             12789|
|    max|            755000|
+-------+------------------+



In [419]:
data_clean.approxQuantile("SalePrice", [0.25, 0.5, 0.75], 0)

[129000.0, 159000.0, 211500.0]

In [420]:
df5 = data_clean.withColumn("Cuartiles", 
      expr("case when SalePrice between 12789 and 129000 then 'Primer_Cuartil' " +
                       "when SalePrice between 129000 and 159000 then 'Segundo_Cuartil' " +
                       "when SalePrice between 159000 and 211500 then 'Tercer_Cuartil' " +
                       "end"))

In [421]:
data_cuartiles = df5.select("Garage Cars", "Cuartiles") \
    .filter(df5.Cuartiles.isNotNull() & df5['Garage Cars'].isNotNull()) \
    .groupBy("Cuartiles", "Garage Cars") \
    .agg(count("Garage Cars")) \
    .orderBy("Cuartiles")

data_cuartiles.show()

+---------------+-----------+------------------+
|      Cuartiles|Garage Cars|count(Garage Cars)|
+---------------+-----------+------------------+
| Primer_Cuartil|          0|               111|
| Primer_Cuartil|          3|                 7|
| Primer_Cuartil|          4|                 1|
| Primer_Cuartil|          5|                 1|
| Primer_Cuartil|          2|               170|
| Primer_Cuartil|          1|               411|
|Segundo_Cuartil|          3|                12|
|Segundo_Cuartil|          4|                 2|
|Segundo_Cuartil|          0|                28|
|Segundo_Cuartil|          1|               279|
|Segundo_Cuartil|          2|               366|
| Tercer_Cuartil|          0|                 7|
| Tercer_Cuartil|          3|                25|
| Tercer_Cuartil|          4|                 4|
| Tercer_Cuartil|          1|                61|
| Tercer_Cuartil|          2|               586|
+---------------+-----------+------------------+



Del analisis realizado anteriorment podemos observar como el primer cuartil del precio se ve representado principalmente por las viviendas con un garage. Luego, en el segundo cuartil del precio se observa que esta representado por casas con dos o una plaza de garage ya que la mayoria de estos se encuentran en esta parte de la distribucion. Por ultimo, podemos observar que el tercer cuartil se encuentra representado principalmente por viviendas con dos plazas de garage

## (COMPLEMENTARIA) Las 10 viviendas con mejores servicios y mejor precio.

### Pista:
#### Calcula las variables: 
- Número de servicios excelentes
- Número de servicios buenos
...

#### Tendréis que tener en cuenta también variables como número de baños Full Bath, cocinas Kitchen AbvGr o dormitorios Bedroom AbvGr. Generando por ejemplo (número de estas variables por encima de media)

#### Finalmente precio de venta